# __eBay Kleinanzeigen Analysis__

In this guided project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

50,000 data points from the full dataset were sampled from the original Kaggle dataset.  DataQuest also dirtied the dataset a bit to more closely resemble what  a scraped dataset.

The data dictionary provided with data is as follows:

* __dateCrawled__ - When this ad was first crawled. All field-values are taken from this date.
* __name__ - Name of the car.
* __seller__ - Whether the seller is private or a dealer.
* __offerType__ - The type of listing
* __price__ - The price on the ad to sell the car.
* __abtest__ - Whether the listing is included in an A/B test.
* __vehicleType__ - The vehicle Type.
* __yearOfRegistration__ - The year in which the car was first registered.
* __gearbox__ - The transmission type.
* __powerPS__ - The power of the car in PS.
* __model__ - The car model name.
* __kilometer__ - How many kilometers the car has driven.
* __monthOfRegistration__ - The month in which the car was first registered.
* __fuelType__ - What type of fuel the car uses.
* __brand__ - The brand of the car.
* __notRepairedDamage__ - If the car has a damage which is not yet repaired.
* __dateCreated__ - The date on which the eBay listing was created.
* __nrOfPictures__ - The number of pictures in the ad.
* __postalCode__ - The postal code for the location of the vehicle.
* __lastSeenOnline__ - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#Read in the data file using specific encoding
autos = pd.read_csv('autos.csv', encoding = 'Windows-1252')

In [3]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


__Initial Observations__

_Data info display:_
* A handful of variables will need new to be converted to other data types
* Column headers need to be standardized (underscores and all lowercase should be used instead of camelcase)
* As many as 10,000 variables are missing in some columns.  Look into these to determine why they're missing and if they can be filled in.


_Data table:_
* Most of the table appears to be in German; I'll need to translate it to English.
* Data in name column is a mess and hard to read
* Prices appear to be in an American format (good, but double check)
* Dates & other timestamps are in good format (but will need to double check)

In [4]:
# Convert the column names from camelcase to snakecase 
columns = autos.columns.copy()
print(columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [5]:
# Dictionary of replacement names
rep_names = {'dateCrawled':'date_crawled',
             'offerType':'offer_type',
             'abtest':'ab_test',
             'vehicleType':'vehicle_type',
             'yearOfRegistration':'registration_year',
             'powerPS':'power_ps',
             'fuelType':'fuel_type',
             'monthOfRegistration':'registration_month',
             'notRepairedDamage':'unrepaired_damage',
             'dateCreated':'ad_created',
             'nrOfPictures':'num_pictures',
             'postalCode':'postal_code',
             'lastSeen':'last_seen'
            }
# Make the replacement
autos.rename(rep_names, axis = 1, inplace = True)
# Check work
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

Changes were made to column headers for the purpose of standardization and clarity.  Now, the information in the column headers is more easily identified and understood with the use of snakecase instead of the previous camelcase.

In [6]:
#Explore the data to understand what can be cleaned and/or salvaged
autos.describe(include = 'all')
# the 'include = 'all'' parameter is added to account for columns with categorical variables

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [7]:
#Convert the dataframe's column names to a list
columns = autos.columns.tolist()

In [8]:
#Define a function that will create a count of all unique items in each column
def count_generator(col_list):
    for each in col_list:
        freq_each = autos[each].value_counts(dropna = False)
        print('Column name: ', each)
        print(freq_each, '\n', '\n')

In [9]:
count_generator(columns)

Column name:  date_crawled
2016-03-30 17:37:35    3
2016-03-09 11:54:38    3
2016-04-04 16:40:33    3
2016-04-02 15:49:30    3
2016-03-19 17:36:18    3
2016-03-05 16:57:05    3
2016-03-22 09:51:06    3
2016-03-21 16:37:21    3
2016-03-27 22:55:05    3
2016-03-30 19:48:02    3
2016-03-08 10:40:35    3
2016-03-14 20:50:02    3
2016-03-12 16:06:22    3
2016-03-16 21:50:53    3
2016-03-25 19:57:10    3
2016-03-21 20:37:19    3
2016-03-23 19:38:20    3
2016-03-23 18:39:34    3
2016-03-29 23:42:13    3
2016-03-10 15:36:24    3
2016-04-02 11:37:04    3
2016-03-11 22:38:16    3
2016-03-14 13:55:55    2
2016-03-19 10:50:31    2
2016-04-02 14:44:49    2
2016-04-04 19:49:16    2
2016-03-24 11:54:49    2
2016-04-04 20:45:44    2
2016-03-19 19:49:39    2
2016-04-04 01:57:49    2
                      ..
2016-03-10 19:57:03    1
2016-03-10 10:46:08    1
2016-04-01 16:54:38    1
2016-03-25 22:56:07    1
2016-03-09 10:54:11    1
2016-04-03 01:57:12    1
2016-03-25 11:56:18    1
2016-03-24 10:54:22    

__Exploratory Notes:__

* Any columns that have mostly one value that are candidates to be dropped
    * Every entry in the data set indicates that no pictures were attached to the ad


* Any columns that need more investigation.
    * The seller column is composed almost entirely of private sales (1 exception)
    * The offer-type column is composed almost entirely of offers, as opposed to 'petitions' (Gesuch) - (1 exception)
    * Some registration_year entries don't make sense ('9000', '1800', '4100', '4800', etc.)
    * There are 1421 entries whose price was \$0 - this seems weird and may be fixable
    * Several others entries in 'price' for very low amounts (i.e. \\$3 (count: 1) - $350 (count: 700+))
    * A few entries in price for exorbitant amounts (\$197,000 - 250,000)
    * There are 5500 entries in the power_ps column with a value of 0 (which is clearly incorrect).  Other extreme (50, 9011) values may be worth investigating as well.
    * There are 5075 values of '0' in the month column (with 1-12 indicating January - December)
    * There are a few entries where the postal codes may be missing a number (4 instead of 5-digit numbers)


* The following columns contain data stored as text which should be cleaned and converted to numeric data:
    * price
    * registration_year
    * power_ps
    * odometer - (move the 'km' to the column name, and convert to numeric data)
    * registration_month

In [10]:
# Renaming column headers to include data that was removed (i.e. '$' and 'km')
autos.rename(columns = {'price':'price_usd'}, inplace = True)
autos.rename(columns = {'odometer':'odometer_km'}, inplace = True)

# Removing non-numeric characters in the price and odometer columns and converting column data to numeric type
autos['price_usd'] = autos['price_usd'].str.replace('$', '')
autos['price_usd'] = autos['price_usd'].str.replace(',', '').astype(int)
autos['odometer_km'] = autos['odometer_km'].str.replace('km', '')
autos['odometer_km'] = autos['odometer_km'].str.replace(',', '').astype(int)

# Check my work
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
date_crawled          50000 non-null object
name                  50000 non-null object
seller                50000 non-null object
offer_type            50000 non-null object
price_usd             50000 non-null int32
ab_test               50000 non-null object
vehicle_type          44905 non-null object
registration_year     50000 non-null int64
gearbox               47320 non-null object
power_ps              50000 non-null int64
model                 47242 non-null object
odometer_km           50000 non-null int32
registration_month    50000 non-null int64
fuel_type             45518 non-null object
brand                 50000 non-null object
unrepaired_damage     40171 non-null object
ad_created            50000 non-null object
num_pictures          50000 non-null int64
postal_code           50000 non-null int64
last_seen             50000 non-null object
dtypes: int32(2), 

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Now, we'll analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.

### __Exploring the Price column data__

In [11]:
#List the number of unique price values in the column
autos['price_usd'].unique().shape

(2357,)

In [12]:
autos['price_usd'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_usd, dtype: float64

In [13]:
#Check the prices listed most frequently
autos['price_usd'].value_counts().sort_index(ascending = True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price_usd, dtype: int64

In [14]:
#Check the prices listed least frequently
autos['price_usd'].value_counts().sort_index(ascending = False).head(200)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
169000      1
163991      1
163500      1
155000      1
151990      1
           ..
47500       3
47499       1
47000       4
46999       1
46990       1
46911       1
46900       3
46800       1
46500       1
46200       1
46000       2
45950       1
45949       1
45900       3
45800       1
45500       4
45000       5
44996       1
44990       1
44900       5
44777       1
44500       1
44499       2
44497       1
44444       1
44200       2
44000       4
43900       5
43500       2
43461       1
Name: price_usd, Length: 200, dtype: int64

In [15]:
# Retrieve the data of cars whose prices were between 340,000 and 99,999,999
autos.query('340000 <= price_usd < 99999999').sort_values('price_usd', ascending = False)
# Modify the 340000 value as needed to view different sets of the data

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
42221,2016-03-08 20:39:05,Leasinguebernahme,privat,Angebot,27322222,control,limousine,2014,manuell,163,c4,40000,2,diesel,citroen,NaN,2016-03-08 00:00:00,0,76532,2016-03-08 20:39:05
27371,2016-03-09 15:45:47,Fiat_Punto,privat,Angebot,12345678,control,NaN,2017,NaN,95,punto,150000,0,NaN,fiat,NaN,2016-03-09 00:00:00,0,96110,2016-03-09 15:45:47
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,privat,Angebot,12345678,control,NaN,2018,manuell,95,v40,150000,6,NaN,volvo,nein,2016-03-08 00:00:00,0,14542,2016-04-06 23:17:31
47598,2016-03-31 18:56:54,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,privat,Angebot,12345678,control,limousine,2001,manuell,101,vectra,150000,3,benzin,opel,nein,2016-03-31 00:00:00,0,4356,2016-03-31 18:56:54
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,privat,Angebot,11111111,test,limousine,1973,manuell,48,escort,50000,3,benzin,ford,nein,2016-03-12 00:00:00,0,94469,2016-03-12 22:45:27
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,privat,Angebot,11111111,test,NaN,1995,NaN,0,NaN,150000,0,NaN,volkswagen,NaN,2016-03-21 00:00:00,0,18519,2016-03-21 14:40:18
11137,2016-03-29 23:52:57,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,privat,Angebot,10000000,control,coupe,1960,manuell,368,NaN,100000,1,benzin,sonstige_autos,nein,2016-03-29 00:00:00,0,73033,2016-04-06 21:18:11
47634,2016-04-04 21:25:21,Ferrari_FXX,privat,Angebot,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60313,2016-04-05 12:07:37
7814,2016-04-04 11:53:31,Ferrari_F40,privat,Angebot,1300000,control,coupe,1992,NaN,0,NaN,50000,12,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60598,2016-04-05 11:34:11
22947,2016-03-22 12:54:19,Bmw_530d_zum_ausschlachten,privat,Angebot,1234566,control,kombi,1999,automatik,190,NaN,150000,2,diesel,bmw,NaN,2016-03-22 00:00:00,0,17454,2016-04-02 03:17:32


In [16]:
# Display the number of cars with prices under $100
autos.query(' price_usd < 100').sort_values('price_usd', ascending = True)

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
24245,2016-03-12 15:57:30,Traum_OPEL_SENATOR_3.0E_2_hand_top_zust.WILDLEDER,privat,Angebot,0,control,limousine,1982,manuell,180,andere,150000,5,NaN,opel,nein,2016-03-12 00:00:00,0,78239,2016-03-14 07:46:42
31144,2016-03-22 12:51:45,Bastlerfahrzeug_Opel_Astra_F,privat,Angebot,0,control,NaN,2017,automatik,75,astra,150000,6,NaN,opel,NaN,2016-03-22 00:00:00,0,24534,2016-03-22 12:51:45
31130,2016-03-17 17:48:19,Tausche_Vectra_C,privat,Angebot,0,control,NaN,2005,NaN,0,vectra,150000,0,NaN,opel,NaN,2016-03-17 00:00:00,0,39387,2016-03-28 21:45:42
31094,2016-03-09 16:55:27,Renault_Laguna_/_Tuev__03.17,privat,Angebot,0,test,kombi,1998,manuell,107,laguna,150000,8,benzin,renault,ja,2016-03-09 00:00:00,0,38640,2016-03-17 19:17:37
31060,2016-03-10 11:37:00,Mercedes_107_SLC_350er_Oldtimer,privat,Angebot,0,test,limousine,1973,automatik,200,sl,150000,1,benzin,mercedes_benz,nein,2016-03-10 00:00:00,0,50189,2016-04-07 05:15:57
31051,2016-03-29 20:51:37,Tausche_ml_Vollauslastung_270_cdi_TÜV_neu,privat,Angebot,0,test,suv,2002,automatik,0,m_klasse,150000,0,diesel,mercedes_benz,ja,2016-03-29 00:00:00,0,6132,2016-04-06 07:17:14
31015,2016-03-26 22:53:22,MB_510_Kipper_mit_Front_anbauplatte,privat,Angebot,0,test,andere,1994,automatik,115,andere,150000,0,benzin,mercedes_benz,nein,2016-03-26 00:00:00,0,30453,2016-03-29 13:42:48
31014,2016-03-16 19:45:25,Volkswagen_Golf_3_1_6__75ps__mit_TÜV,privat,Angebot,0,test,NaN,2016,manuell,75,golf,150000,0,benzin,volkswagen,nein,2016-03-16 00:00:00,0,29664,2016-04-07 04:45:55
31006,2016-04-04 22:57:48,Honda_crx_del_sol_zum_tausch_kein_Verkauf_letz...,privat,Angebot,0,test,cabrio,1995,manuell,125,cr_reihe,150000,0,benzin,honda,NaN,2016-04-04 00:00:00,0,32339,2016-04-07 02:17:19
31003,2016-03-20 16:53:26,Astra_G_Coupe_2.2liter,privat,Angebot,0,test,coupe,2001,NaN,147,NaN,150000,2,benzin,opel,nein,2016-03-20 00:00:00,0,65599,2016-03-23 14:18:16


In [17]:
# Create an index of the rows we want to delete and use it to remove the outlying car prices (with exception of the two Ferraris in this range)
indexNames = autos[ (autos['price_usd'] >= 349999) | (autos['price_usd'] < 100)]
indexNames1 = indexNames[ (indexNames['name'] != 'Ferrari_FXX')]
indexNames2 = indexNames1[ (indexNames1['name'] != 'Ferrari_F40')].index
# Length should decrease for each condition above
print(len(indexNames), len(indexNames1), len(indexNames2))
# 
autos.drop(indexNames2, inplace = True)

1777 1776 1775


In [18]:
#Check my work (should have only two cars with prices > $350,000 - the Ferrari FXX and Ferrari F40 and no cars with prices < $100)
autos.sort_values('price_usd', ascending = False)

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen
47634,2016-04-04 21:25:21,Ferrari_FXX,privat,Angebot,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60313,2016-04-05 12:07:37
7814,2016-04-04 11:53:31,Ferrari_F40,privat,Angebot,1300000,control,coupe,1992,NaN,0,NaN,50000,12,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60598,2016-04-05 11:34:11
14715,2016-03-30 08:37:24,Rolls_Royce_Phantom_Drophead_Coupe,privat,Angebot,345000,control,cabrio,2012,automatik,460,NaN,20000,8,benzin,sonstige_autos,nein,2016-03-30 00:00:00,0,73525,2016-04-07 00:16:26
34723,2016-03-23 16:37:29,Porsche_Porsche_911/930_Turbo_3.0__deutsche_Au...,privat,Angebot,299000,test,coupe,1977,manuell,260,911,100000,7,benzin,porsche,nein,2016-03-23 00:00:00,0,61462,2016-04-06 16:44:50
35923,2016-04-03 07:56:23,Porsche_911_Targa_Exclusive_Edition__1_von_15_...,privat,Angebot,295000,test,cabrio,2015,automatik,400,911,5000,6,benzin,porsche,nein,2016-04-03 00:00:00,0,74078,2016-04-03 08:56:20
12682,2016-03-28 22:48:01,Porsche_GT3_RS__PCCB__Lift___grosser_Exklusiv_...,privat,Angebot,265000,control,coupe,2016,automatik,500,911,5000,3,benzin,porsche,nein,2016-03-28 00:00:00,0,70193,2016-04-05 03:44:51
47337,2016-04-05 10:25:38,BMW_Z8_roadster,privat,Angebot,259000,test,cabrio,2001,manuell,400,z_reihe,20000,6,benzin,bmw,nein,2016-04-05 00:00:00,0,61462,2016-04-05 12:07:32
38299,2016-03-28 22:25:25,Glas_BMW_mit_Wasser,privat,Angebot,250000,test,NaN,2015,NaN,0,x_reihe,5000,0,NaN,bmw,NaN,2016-03-28 00:00:00,0,60489,2016-03-28 22:25:25
37840,2016-03-21 10:50:12,Porsche_997,privat,Angebot,220000,test,coupe,2008,manuell,415,911,30000,7,benzin,porsche,nein,2016-03-21 00:00:00,0,69198,2016-04-06 04:46:14
40918,2016-03-20 18:40:05,Porsche_911_991_GT3_RS,privat,Angebot,198000,test,coupe,2015,automatik,500,911,5000,6,benzin,porsche,nein,2016-03-20 00:00:00,0,51491,2016-03-21 21:46:36


### __Exploratory results__
There are a handful of entries with prices higher than I would expect for used cars.  I took a look at the individual cars to see if the prices appear accurate.  For anything above \\$350,000, I believe that that the entries are bogus.  At \\$350,000 and below, yes, the prices are high, but these are for newer (still used) cars such as a 2016 Porche 911 and a 2012 Rolls Royce Phantom.  Thus, I've removed entries (outliers) with a price higher than \\$350,000.

As for the low prices, there are 1762 values below \$100.  I've decided to drop these as well as there's no justification for such a low price (unless donations were allowed on this particular eBay site).

__Sidenote on 'name' column__: I did notice that some of the names were entered somewhat incorrectly (i.e. Porsche_991 or 997 (which are internal company designations), instead of common-name, Porsche 911).  This just means that we'll have to standardize the names first if we want to carry out any analysis involving the data in that column.  I believe that the model is occassionally listed correctly in the 'model' column.

__Sidenote on 'power_ps' column__: In my cursory check of these high priced cars, it appears that their corresponding horsepower listed in the power_ps column is accurate.

### __Exploring the Odometer column data__

In [19]:
#List the number of unique odometer values in the column
autos['odometer_km'].unique().shape

(13,)

In [20]:
# Display the basic descriptive statistics of the odometer_km column of the dataset
autos['odometer_km'].describe()

count     48225.000000
mean     125917.573872
std       39544.440846
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [21]:
# Display the unique values and counts of those values
autos['odometer_km'].value_counts().sort_index(ascending = False)

150000    31212
125000     5037
100000     2101
90000      1733
80000      1412
70000      1214
60000      1153
50000      1010
40000       814
30000       777
20000       757
10000       245
5000        760
Name: odometer_km, dtype: int64

In [22]:
# Check how many cars in the dataset have an odometer reading less than 5000km and more than 150000km
autos.query('150000 < odometer_km < 5000').sort_values('odometer_km', ascending = True)

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen


### __Exploratory Results (Odometer_km)__

The odometer_km column has already been converted to a numeric datatype.  All values fall between 5000 and 150000, which is a reasonable range, with half of all values falling at 150,000km.  Realistically, these seem to be bins, with this last group being '150000+'.  

I don't see any outliers in the odometer_km column, so I'll be leaving it as-is for the moment.

I'll now take a look at the columns with datetime-like data.

In [23]:
autos[['date_crawled', 'registration_year', 'registration_month', 'ad_created', 'last_seen']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48225 entries, 0 to 49999
Data columns (total 5 columns):
date_crawled          48225 non-null object
registration_year     48225 non-null int64
registration_month    48225 non-null int64
ad_created            48225 non-null object
last_seen             48225 non-null object
dtypes: int64(2), object(3)
memory usage: 2.2+ MB


In [24]:
# Print the 3 columns with 'object' data types
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [25]:
# ***Not actually using this anymore, but wanted to keep it for reference***

# As the first 10 characters of each line belong to the year, month, date, we can use a string slice to obtain that data
    # print(autos['date_crawled'].str[:10])
# Attempting to convert them to datetime types instead first though
# autos['date_crawled'] = pd.to_datetime(autos['date_crawled'], format = '%Y-%m-%d')
# autos['ad_created'] = pd.to_datetime(autos['ad_created'], format = '%Y-%m-%d')
# autos['last_seen'] = pd.to_datetime(autos['last_seen'], format = '%Y-%m-%d')


In [26]:
# Save the year, month, day information to a new column for each column of interest
autos['date_crawled_date'] = autos['date_crawled'].str[:10]
autos['ad_created_date'] = autos['ad_created'].str[:10]
autos['last_seen_date'] = autos['last_seen'].str[:10]

In [27]:
# Calculates the distribution of each date in the column and displays it as a percentage
autos['date_crawled_date'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025360
2016-03-06    0.014038
2016-03-07    0.036060
2016-03-08    0.033178
2016-03-09    0.033012
2016-03-10    0.032286
2016-03-11    0.032597
2016-03-12    0.036910
2016-03-13    0.015677
2016-03-14    0.036661
2016-03-15    0.034318
2016-03-16    0.029466
2016-03-17    0.031498
2016-03-18    0.012898
2016-03-19    0.034733
2016-03-20    0.037802
2016-03-21    0.037201
2016-03-22    0.032888
2016-03-23    0.032286
2016-03-24    0.029445
2016-03-25    0.031498
2016-03-26    0.032307
2016-03-27    0.031104
2016-03-28    0.034961
2016-03-29    0.034111
2016-03-30    0.033738
2016-03-31    0.031851
2016-04-01    0.033696
2016-04-02    0.035604
2016-04-03    0.038611
2016-04-04    0.036579
2016-04-05    0.013064
2016-04-06    0.003173
2016-04-07    0.001389
Name: date_crawled_date, dtype: float64

There's not much insight here other than the fact that the distribution is fairly even (roughly 3% per day over 33 days) with a few exceptions.  It may be interesting to find out if there's a pattern in those days with a 1.5% distribution.

In [28]:
autos['ad_created_date'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033095
2016-03-10    0.031996
2016-03-11    0.032908
2016-03-12    0.036744
2016-03-13    0.017045
2016-03-14    0.035293
2016-03-15    0.034049
2016-03-16    0.029964
2016-03-17    0.031166
2016-03-18    0.013582
2016-03-19    0.033613
2016-03-20    0.037864
2016-03-21 

The 48225 ads were created between June 11, 2015 and April 7, 2016.  The numbe of ads created appears to increase over time.  A graph would help us confirm this trend though.

In [29]:
autos['last_seen_date'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.001078
2016-03-06    0.004313
2016-03-07    0.005433
2016-03-08    0.007320
2016-03-09    0.009580
2016-03-10    0.010638
2016-03-11    0.012400
2016-03-12    0.023784
2016-03-13    0.008875
2016-03-14    0.012628
2016-03-15    0.015863
2016-03-16    0.016444
2016-03-17    0.028097
2016-03-18    0.007320
2016-03-19    0.015759
2016-03-20    0.020653
2016-03-21    0.020550
2016-03-22    0.021358
2016-03-23    0.018580
2016-03-24    0.019762
2016-03-25    0.019098
2016-03-26    0.016672
2016-03-27    0.015531
2016-03-28    0.020840
2016-03-29    0.022291
2016-03-30    0.024697
2016-03-31    0.023826
2016-04-01    0.022851
2016-04-02    0.024883
2016-04-03    0.025132
2016-04-04    0.024531
2016-04-05    0.125101
2016-04-06    0.221960
2016-04-07    0.132151
Name: last_seen_date, dtype: float64

The last_seen column appears to hold the same range as the date_crawled column which is to be expected as the crawlers were only implemented in the last month before the data was collected.  These frequencies are much lower though than the date_crawled frequencies, which I think would indicate that the ads were taken down (hopefully due to being sold).

In [30]:
autos['registration_year'].describe()

count    48225.000000
mean      2004.730492
std         87.896481
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The average year is 2004, which sounds reasonable.  The bottom 25% of cars was registered in 1999, with the bottom 75% of cars being registered during or before 2008. 

It is a problem that the minimum is 1000 and the max is 9999 as these are not valid years.  I'll either need to extrapolate the date somehow or (more likely) remove those rows.  This will hopefully help with the odd standard deviation too.

While some dealers do list cars for sale as early as spring the year before the model year, these will be predominantly new cars.  Consequently, I'm going to assume that the 1383 cars registered in 2017 were incorrectly entered.  Anything after 2017 is definitely not feasible and will also be removed (as will, of course, those with a year before 1900).

In [31]:
# Two methods to identify the values outside of the 1900-2016 range.  I'm utilizing the second because it's more concise and quicker

# Method 1:
# Create a dictionary to store the results
# problem_entries = {}

# Counting the the number of listings with cars that fall outside the 1900 - 2016 interval to further investigate the situation
# for item in autos['registration_year']:
#     if (item < 1900) | (item > 2016):
#         if item in problem_entries:
#             problem_entries[item] += 1
#         else:
#             problem_entries[item] = 1
            
# Print out the results
# for k, v in problem_entries.items():
#     print(k, ':', v)

# # Print the total number of entries with a registration year outside the 1900-2016 range
# sum(problem_entries.values())

# Method 2 (note that 'problem_entries' doesn't refer to the same object in each method): 
problem_entries = autos[(autos['registration_year'] < 1900) | (autos['registration_year'] > 2016)].index
problem_entries
print("You started with {} entries".format(len(autos)))
print("You have {} problem entries".format(len(problem_entries)))


You started with 48225 entries
You have 1872 problem entries


In [32]:
# Remove the values contained in the dictionary from the main dataframe
autos.drop(problem_entries, inplace = True)
# Print the new length to confirm the proper number of values were dropped (compared to the lengths printed above)
len(autos)

46353

In [33]:
# Calculating the new distribution of the registration_year column and the descriptive statistics
autos['registration_year'].value_counts(normalize=True)

2000    0.066964
2005    0.062801
1999    0.062110
2004    0.058227
2003    0.058098
2006    0.057580
2001    0.056717
2002    0.053438
1998    0.050482
2007    0.049037
2008    0.047678
2009    0.044873
1997    0.041529
2011    0.034906
2010    0.034237
1996    0.029232
2012    0.028218
2016    0.025910
1995    0.025737
2013    0.017280
2014    0.014282
1994    0.013505
1993    0.009061
2015    0.008198
1992    0.007939
1991    0.007292
1990    0.007162
1989    0.003689
1988    0.002869
1985    0.002006
          ...   
1966    0.000475
1976    0.000453
1969    0.000410
1975    0.000388
1965    0.000367
1964    0.000259
1963    0.000173
1961    0.000129
1959    0.000129
1962    0.000086
1956    0.000086
1937    0.000086
1958    0.000086
1955    0.000043
1954    0.000043
1957    0.000043
1951    0.000043
1910    0.000043
1934    0.000043
1941    0.000043
1927    0.000022
1929    0.000022
1950    0.000022
1931    0.000022
1948    0.000022
1938    0.000022
1939    0.000022
1953    0.0000

In [34]:
autos['registration_year'].describe()

count    46353.000000
mean      2002.939335
std          7.127006
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

The minimums, maximums, and standard deviation all appear much more normal now that we've droppped the outliers.  A large percentage of the cars were registered in the late '90s and early 00s (until about 2010).

In [35]:
# Gather the unique elements in the brand column and return them with the count of each brand 
unique_brands, brand_counts = np.unique(autos['brand'], return_counts=True)
# Combine the two numpy arrays into a dictionary, convert it to a dataframe, and reset the index (not sure if the zip is necessary)
brands = pd.DataFrame.from_dict(dict(zip(unique_brands, brand_counts)), orient='index').reset_index()

In [36]:
# Rename the columns
brands.rename(columns = {'index':'brand', 0 :'count'}, inplace=True)

In [37]:
# Add a column for brand frequency
brands['brand_freq'] = brands['count'] / brands['count'].sum()
brands.sort_values('brand_freq', ascending = False)

,brand,count,brand_freq
38,volkswagen,9799,0.211399
2,bmw,5107,0.110176
24,opel,4971,0.107242
20,mercedes_benz,4480,0.096650
1,audi,4022,0.086769
10,ford,3237,0.069834
27,renault,2182,0.047074
25,peugeot,1384,0.029858
9,fiat,1187,0.025608
30,seat,846,0.018251


The bulk of the data is contained within roughly the first six brands.  However, to err on the side of caution (of having too much data rather than too little), I'm going to select all brands with at least a 2% share for further evaluation.

In [38]:
# Create a filter series to pull out the brands with a brand_freq > 0.02
brands_filter = (brands['brand_freq'] > 0.02)
# Use the filter to process the process the new list
brands_top = brands[brands_filter.values].sort_values('brand_freq', ascending = False)
# Check my work
brands_top

,brand,count,brand_freq
38,volkswagen,9799,0.211399
2,bmw,5107,0.110176
24,opel,4971,0.107242
20,mercedes_benz,4480,0.096650
1,audi,4022,0.086769
10,ford,3237,0.069834
27,renault,2182,0.047074
25,peugeot,1384,0.029858
9,fiat,1187,0.025608


In [39]:
# Create an empty dictionary to hold the aggregated data
brands_prices = {}

# Loop over the brands from brands_top, assigning the brands (key) and their mean prices (value) to the dictionary
for i in brands_top['brand']:
    brand_only = autos[autos['brand'] == i]
    mean_price = brand_only['price_usd'].mean()
    brands_prices[i] = int(mean_price)
    
# Print the dictionary
brands_prices

{'volkswagen': 5436,
 'bmw': 8381,
 'opel': 3005,
 'mercedes_benz': 8672,
 'audi': 9380,
 'ford': 3779,
 'renault': 2496,
 'peugeot': 3113,
 'fiat': 2836}

We can see here that Audi, Mercedez Benz, and BMW top the list of average prices by brand with Volkswagen falling roughly \\$3000 behind on average.  The rest of the group falls another $2000 behind Volkswagen on average.  It's interesting that the cars fit pretty neatly into three nice bins.

Let's see if there is any relationship between mean mileage and mean price for each of these brands

In [40]:
# Create an empty dictionary to hold the newly aggregated data
brands_mileage = {}

# Loop over the brands from brands_top, assigning the brands (key) and their mean mileage (value) to the dictionary
for i in brands_top['brand']:
    brand_only = autos[autos['brand'] == i]
    mean_mileage = brand_only['odometer_km'].mean()
    brands_mileage[i] = int(mean_mileage)
    
# Print the dictionary
brands_mileage

{'volkswagen': 128799,
 'bmw': 132695,
 'opel': 129384,
 'mercedes_benz': 131025,
 'audi': 129245,
 'ford': 124277,
 'renault': 128281,
 'peugeot': 127127,
 'fiat': 116950}

In [41]:
#Convert the brands_prices and brands_mileage dictionaries to Pandas series (called 'bmp_series' and 'bmm_series' respectively)
bmp_series = pd.Series(brands_prices)
bmm_series = pd.Series(brands_mileage)

# Create a dataframe for the bmp object
bmp = pd.DataFrame(bmp_series, columns = ['mean_price'])
# Assign the bmm series to the new bmp dataframe
bmp['mean_mileage'] = bmm_series

# Pretty print the dataframe
bmp.sort_values('mean_price', ascending = False)

,mean_price,mean_mileage
audi,9380,129245
mercedes_benz,8672,131025
bmw,8381,132695
volkswagen,5436,128799
ford,3779,124277
peugeot,3113,127127
opel,3005,129384
fiat,2836,116950
renault,2496,128281


The mean mileages for each brand are all close enough to eachother that no patterns are clearly evident.  As the literature shows that mileage is often the one of the greater predictors of a car's value, without further investigation, we may suspect that the difference in cost between brands is due to the value inherent in the brands themselves.  

Fortunately, we have other fields such as engine power, gearbox, and unrepaired damage that could all serve as a viable determiner of the car's mean price.

Next, I'll take a look at the categorical data that uses german words.  I'll translate these words and map them back to the dataset so it's easier to understand.

In [42]:
autos.head()

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,...,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen,date_crawled_date,ad_created_date,last_seen_date
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,...,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54,2016-03-26,2016-03-26,2016-04-06
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,...,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08,2016-04-04,2016-04-04,2016-04-06
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,...,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37,2016-03-26,2016-03-26,2016-04-06
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,...,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28,2016-03-12,2016-03-12,2016-03-15
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,...,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50,2016-04-01,2016-04-01,2016-04-01


Columns with german-language data are:
* name
* seller
* offer_type
* vehicle_type
* gearbox
* fuel_type
* unrepaired_damage

The name column is a project in and of itself, so we'll start with the other columns.

In [43]:
# Find the unique values for each of the above columns and convert the values to a list (for ease in translation):
sellers = autos['seller'].unique()
sellers = list(sellers)

In [44]:
offer_types = autos['offer_type'].unique()
offer_types = list(offer_types)

In [45]:
vehicle_types = autos['vehicle_type'].unique()
vehicle_types = list(vehicle_types)

In [46]:
gearboxes = autos['gearbox'].unique()
gearboxes = list(gearboxes)

In [47]:
fuel_types = autos['fuel_type'].unique()
fuel_types = list(fuel_types)

In [48]:
unrepaired_damage = autos['unrepaired_damage'].unique()
unrepaired_damage = list(unrepaired_damage)

In [49]:
# Utilize the googletrans library to translate german words from the lists above into english and add them to a dictionary
from googletrans import Translator
# Create an instance of translator to utilize the embedded API (?)
translator = Translator()
# Create a blank dictionary to hold translations

'''This translation function (trans_func) reads in any number of lists of words and assigns the original word 
and its english translation to the dictionary 'translations'.  It's only been tested with multiple lists of words 
from a single language.
'''
def trans_func(*args):
    translations = {}  #Create a dictionary to hold the translations
    args = list([val for sublist in args for val in sublist])  #Flatten the list
    for item in args:
        if type(item) != str: #Remove any items in the lists that are not strings
            args.remove(item)
        else:
            translations.update({item:translator.translate(item, dest='en').text},)  # Initiate the translation of the lists
    return translations

# Translate all german words contained in the lists above
translations = trans_func(sellers, offer_types, vehicle_types, gearboxes, fuel_types, unrepaired_damage)

I noticed that a couple of these translations - particularly 'cabrio' (convertible), 'manuell' (manual), and 'electro' (electric) did not translate properly.  I know that the German language typically capitalizes nouns, and thus was suspicious of a few of the translations.  After manually translating these oddities, I confirmed that the translator library does not translate 'cabrio' the same as '<b>C</b>abrio'.  Fortunately, I can make some small changes to the dictionary, and then map the changes to the original dataset for updating.

In [50]:
# Update the dictionary with corrected the three manual translations
translations.update({'kombi':'convertible', 'elektro':'electric', 'manuell':'manual'})
translations

{'privat': 'private',
 'gewerblich': 'commercial',
 'Angebot': 'offer',
 'bus': 'bus',
 'limousine': 'limousine',
 'kleinwagen': 'small car',
 'kombi': 'convertible',
 'suv': 'suv',
 'cabrio': 'cabrio',
 'andere': 'Others',
 'manuell': 'manual',
 'automatik': 'automatic',
 'benzin': 'gasoline',
 'diesel': 'diesel',
 'hybrid': 'hybrid',
 'elektro': 'electric',
 'nein': 'No'}

In [51]:
# Replace german words from the 3rd column ('seller') to the last wtih corresponding translations
autos[2:].replace(translations).head()

# I didn't use the replace method with the entire dataset in the event that some of the names included the same words

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,...,fuel_type,brand,unrepaired_damage,ad_created,num_pictures,postal_code,last_seen,date_crawled_date,ad_created_date,last_seen_date
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,private,offer,8990,test,limousine,2009,manual,102,...,gasoline,volkswagen,No,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37,2016-03-26,2016-03-26,2016-04-06
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,private,offer,4350,control,small car,2007,automatic,71,...,gasoline,smart,No,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28,2016-03-12,2016-03-12,2016-03-15
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,private,offer,1350,test,convertible,2003,manual,0,...,gasoline,ford,No,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50,2016-04-01,2016-04-01,2016-04-01
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,private,offer,7900,test,bus,2006,automatic,150,...,diesel,chrysler,NaN,2016-03-21 00:00:00,0,22962,2016-04-06 09:45:21,2016-03-21,2016-03-21,2016-04-06
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,private,offer,300,test,limousine,1995,manual,90,...,gasoline,volkswagen,NaN,2016-03-20 00:00:00,0,31535,2016-03-23 02:48:59,2016-03-20,2016-03-20,2016-03-23


The name column has quite a lot of data in it.  Unfortunately, the data is very mixed up, containing a combination of brand, model, engine size, & other options (i.e. heated seats) in the field.  In the following few cells, though, I'll take a look at the unqiue values to see if we can utilize the data.

In [52]:
# Create two unique lists (one for unique names, one for the count of the unique names)
unique, counts = np.unique(autos['name'], return_counts=True)
# Zip the lists together, convert the dictionary to a dataframe, and sort the dataframe
df = dict(zip(unique, counts))
df = pd.DataFrame.from_dict(df, orient = 'index').sort_values(0, ascending = False)
df

,0
Volkswagen_Golf_1.4,75
BMW_316i,75
Ford_Fiesta,74
BMW_318i,72
Volkswagen_Polo,72
BMW_320i,71
Opel_Corsa,68
Renault_Twingo,65
Volkswagen_Golf,57
Opel_Corsa_1.2_16V,56


In [53]:
# Count the number of entries in the name column of the dataset containing the word 'golf'
name_count_golf = 0
for name in autos['name']:
    if 'Golf' in name:
        name_count_golf += 1
print('There are {} entries with the word ''golf'' in the name column.'.format(name_count_golf))       

There are 3583 entries with the word golf in the name column.


There are roughly 25+ unique entries (and 3583 total entries) with the word 'golf' (in reference to the Volkswagen Golf car) in the name field.  This is one example of the variety of ways in which car names were added to this field.  Seeing as we already have model and brand information, I'm going to conclude this cleaning, believing that the work to clean the name column, at the moment, is not worth the marginal value it would add to the dataset.

I will, however, continue the analysis to answer a few more questions of interest, namely:
* Find the most common brand/model combinations
* Are there any patterns correlating average prices and milage?
* How much cheaper are cars with damage than their non-damaged counterparts?

In [54]:
# Create a new dataframe by grouping the columns we want from the original dataset and aggregating the values unique to both columns
df_unique = autos.groupby(['brand', 'model']).size().reset_index(name='count')
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 3 columns):
brand    290 non-null object
model    290 non-null object
count    290 non-null int64
dtypes: int64(1), object(2)
memory usage: 6.9+ KB


In [55]:
# Sort the values by count for the top 10 entries
df_unique.sort_values('count', ascending=False).head(10)

,brand,model,count
267,volkswagen,golf,3684
23,bmw,3er,2602
273,volkswagen,polo,1592
182,opel,corsa,1568
271,volkswagen,passat,1345
179,opel,astra,1337
13,audi,a4,1226
141,mercedes_benz,c_klasse,1135
24,bmw,5er,1123
144,mercedes_benz,e_klasse,953


Here, we have the top to most common brand-model combinations in the dataset.  It's interesting to note that the Volkswagen Golf has roughly 100 more entries by this count than my last count, indicating some discrepancy between the data in the 'name' column and that in the 'brand' & 'model' columns.

It's also interesting to note that Volkswagen has three of the top five spots on this list.  This matches up with our earlier analysis on price.

In [73]:
unique, counts = np.unique(autos['odometer_km'], return_counts=True)
unique_odometer = dict(zip(unique, counts))
unique_odometer = pd.DataFrame.from_dict(unique_odometer, orient = 'index')
unique_odometer.reset_index(inplace = True)
unique_odometer.rename(columns = {'index':'odometer_km', 0:'count'})

,odometer_km,count
0,5000,711
1,10000,233
2,20000,739
3,30000,757
4,40000,796
5,50000,991
6,60000,1126
7,70000,1184
8,80000,1372
9,90000,1672


To be continued...